# Setup 

In [ ]:
from pathlib import Path
import subprocess
import datetime
from datetime import date
import re
import os
from importlib import reload
import json
import time

import pandas as pd
import numpy as np
from IPython.core.debugger import Pdb; ipdb = Pdb()
from heudiconv import utils, parser

# don't really need dask (especially distributed),
#  it can be useful for converting larger datasets though
from dask.distributed import Client, LocalCluster
from dask import delayed

import heudiconv_helpers.helpers # this is a module so can be easily reloaded upon modification

def run_cmd(cmd):
    """Run a string that represents a command.
    Generating command strings and executing them is 
    a quick and dirty way of flexibly generating commands
    in python and executing them on different compute
    infrastructures.
    """
    import subprocess
    pp = subprocess.run(cmd,shell=True,stdout=subprocess.PIPE,stderr= subprocess.PIPE)
    print([v.split('//')[-1] for v in pp.stderr.decode('utf-8').splitlines() ])
    return pp

# Set pandas display settings
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('max_colwidth',500)

In [ ]:
n_workers = 2 # EDIT: number of workers to use with Dask.
client = Client(processes = False,
    n_workers= n_workers,
    threads_per_worker=1
    )

In [ ]:
# Define an version string to append to filenames in order to distinguish
# between different analyses.
analysis_version = "2018_08" # EDIT

# Define a project directory. If all analysis file and
# data are container within this directory then all files
# can be defined relative to this. This setup makes it easier
# to handle paths and to move the project between hosts as required
project_dir_absolute = Path('.').absolute() # EDIT: needs to be pathlib.Path object

In [ ]:
# This cell changes to the notebooks ipython kernel
# to the project directory. This enables tab completion
# of all paths within the project which makes things
# go more smoothly and avoids errors.
%pwd
%cd {project_dir_absolute.as_posix()}
%pwd

In [ ]:
# This cell allows lots of customization. In general, the only things
# that are useful to change are marked with the "EDIT" tag. The rest is 
# defined here to increase transparency but should work if left alone.

# Define the directory to search through to gather dicoms for the 
# conversion
dicoms_dir = Path('data/dicoms') # EDIT and place dicoms at this location

scripts_dir  = Path('analysis_files/heudiconv_files')

# Define the path to the heudiconv heuristics script. This script enables one
# to map from dicom metadata to the bids directory specification using some rules
# written in python
heuristics_script = scripts_dir.joinpath('heuristics/heuristics_file_' + analysis_version + '.py')

# Specify the path to a software container image with heudiconv. This can be 
# found on docker hub at nipy/heudiconv. If a conversion to singularity is 
# required you can use https://github.com/singularityware/docker2singularity
sing_image = scripts_dir.joinpath('path_to_heudiconv_image.img')

# If not using compressed dicom directories a slightly differnt syntax
# is required for heudiconv
dicom_extension = '.tar.gz' # EDIT

# When heudiconv is run with a software container, the project 
# directory is mounted into the container at the mount point /data.
#  The output is written to outdir (a path starting with /data), which
# is defined below:
outdir = Path("/data/bids_" + analysis_version)

# The output of heudiconv will have a different path on the local filesystem:
output_of_heudiconv = Path(outdir.as_posix().replace('/data/','')) 

# In order to define the heuristics for heudiconv to use on a dataset,
# a generic run of heudiconv is typically required. This uses a default
# heuristics file that captures all scans and extracts their metadata.
# This metadata is essential to define the mapping as described later.
outdir_gen = Path(outdir.as_posix() + '_generic')
output_of_heudiconv_gen = Path(outdir_gen.as_posix().replace('/data/',''))


logdir = scripts_dir.joinpath('conversion_logs')
symlinked_dicoms = dicoms_dir.absolute().with_name('symlinked_dicoms')


# Recursively create necessary directories for analysis files.
for directory in [symlinked_dicoms,scripts_dir,logdir]:
    if not directory.exists():
        os.makedirs(directory,exist_ok=True)

        
# Create a dictionary to keep track of the different versions of
# a conversion. If it already exists in memory it is just written
# to disk:
conversion_dict_pkld = scripts_dir.joinpath('conversion_dict.pklz')

if 'conversion_dict' in locals():
    conversion_dict.to_pickle(conversion_dict_pkld)
else:
    if conversion_dict_pkld.exists():
        conversion_dict = pd.read_pickle(conversion_dict_pkld)
    else:
        conversion_dict = pd.Series({})

# The path specified below is used to mount the heudiconv source code
# into the software container to allow a more conveneient method of
# debugging any problems.
heudiconv_source_code = "path_to_heudiconv_source_code" # EDIT if debugging is required
    
# Define a path used to a tsv that is created by the notebook to generate
# an arbitrary shift of all acquisition dates for each subject
acq_date_time_offset = logdir.with_name('acquisition_date_time_offset.tsv')

# If available specify a key to map patient ids specified in the dicom
# filenames to an anomyized id
patient_key_path = scripts_dir.joinpath('patient_key.pklz') # EDIT

# A path to record the mapping once heudiconv is run
info_mapping_csv = scripts_dir.joinpath(f'seqinfo_mapping_{analysis_version}.csv')

acq_date_time_offset = scripts_dir  /'acquisition_date_time_offset.pklz'

## Conda environment used

The conda environment used for this analysis can be recreated using the above yml file and the command :

# Generating mapping 

The following example provides an example in which the required dataframe "df_dicoms" is created. The paths to all dicoms are contained in the column "dicom_path" and the patient id and  date of scan is extracted from the scan name. The date is used to sort the scans so that the session numbers assigned in the next cell are in the same order as the scans are acquired. Custom code to generate the subject and session ids can of course replace these two cells.

In [ ]:
df_dicoms = pd.DataFrame({'dicom_path' : [p.as_posix() for p in dicoms_dir.glob('*' + dicom_extension)]})
df_dicoms = pd.concat([df_dicoms,
                       df_dicoms.dicom_path.str.extract(
                              '.*-(?P<patient_id>[0-9]{,8})-(?P<date>[0-9]{8})-*-.*.tgz',expand=True)],
                       axis = 1)

df_dicoms.head()

In [ ]:
def add_bids_ses(df):
    df = df.assign(bids_ses = ['{num:02d}'.format(num = 1 + i) for i in range(len(df))])
    return df

  
df_bids = df_dicoms.apply(
    lambda row: heudiconv_helpers.helpers.gen_bids_subj(
        row,patient_key_path,generate_keys=True),axis = 1)
df_bids = (df_bids.sort_values(['bids_subj','date']).
           groupby(['bids_subj'],as_index = False).
           apply(add_bids_ses)
           ).reset_index(drop=True)

In [ ]:
len(df_bids)

## Create symlinks for heudiconv

There are different ways of using heudiconv. The use-pattern described here requires the dicom tars to have the subject and session in the file name. This is handled by creating a directory of symlinks with the appropriate filenames.

In [ ]:
df_bids['symlink_names'] = df_bids.apply(lambda row: heudiconv_helpers.helpers.get_symlink_name(row),axis=1)
df_bids = df_bids.assign(symlink_path = lambda df: [symlinked_dicoms.joinpath(p) for p in df.symlink_names])
df_bids['symlink_template'] = df_bids.apply(hh.make_symlink_template,project_dir_absolute= project_dir_absolute.as_posix(),axis=1)
df_bids['dicom_template'] =  df_bids.symlink_template

In [ ]:
df_bids.apply(lambda row: hh.make_symlink(row,project_dir_absolute,overwrite_previous=True), axis=1);

##  Run heudiconv without conversion

The first run of heudiconv, described here as "generic", uses a heudiconv heuristics script designed to extract the metadata for all scans in the dicoms. This provides information for subsequent runs during which the heuristics more accurately map each scan onto the desired file layout and naming convention, in our case BIDS. This subsequent run also allows us to be more picky with which scans we choose to use for the final dataset.

### Generate parallel commands

In [ ]:
reload(heudiconv_helpers)
df_sing = df_bids
df_sing['cmd'] = df_bids.apply(
    lambda row:
    heudiconv_helpers.helpers.make_heud_call(row = row,
                           project_dir = project_dir_absolute,
                           output_dir=outdir_gen,
                           conversion = False,
                           container_image = sing_image,
                           bind_path  = "/gs3,/gs4,/gs5,/gs6,/gs7,/gs8,/gs9,/gs10,/gs11,/spin1,/scratch,/fdb",
                           scratch_dir = "/lscratch/$SLURM_JOB_ID"),
        axis = 1)


conversion_dict['generic'] = (Path(scripts_dir)
                              heudiconv_helpers.helpers.joinpath(
                                  'heudiconv_generic_swarm_' + analysis_version + '.cmd'))

# not all commands resolve to a single dicom so getting unique ones before writing swarm
conversion_dict['generic'].write_text('\n'.join(df_sing.cmd.drop_duplicates())) 
print(conversion_dict['generic'].read_text().splitlines()[1])

Example of running the commands on a server:

Example of running the command on nih cluster:

## Heudiconv with custom heuristics script

###  Check heuristics

Heudiconv requires a heuristics file (created in the next section) in order to map the dicom files' metadata to the bids output structure. This is documented at the  nipy/heudiconv github repository. The file contains two main parts:
1. Templates create using the "create_key" function that specify where each run type belongs
2. The specification of the heuristic to categorise each run in the dicom tar.

The template is quite stereotyped and the examples on github are useful in figuring out how to write them.

The heuristic for categorising the runs is a little more challenging. Often the series description from the dicom header can be enough to categorise the scans

### An example heuristics file

This example heuristics file can be a little confusing. The subsequent walks through how to test and debug it though so fear not! Following that section you can fully debug the heuristics specified before you use it with heudiconv.

In [ ]:
heuristics_script

In [ ]:
%%writefile anal/heudiconv_files/heuristics/fmrif_heuristics2018_08_28.py
# modify the file path entered in the line above to match the 
# current heuristics script path
# sub-<participant_label>[_ses-<session_label>]_task-<task_label>[_acq-<label>][_rec-<label>][_run-<index>][_echo-<index>]_bold.nii[.gz]
import os
from collections import namedtuple
import logging
lgr = logging.getLogger('heudiconv')


def infotoids(seqinfos, outdir):
    # This is included from reproin. Look there for examples on 
    # how to use it.
    lgr.info("Processing sequence infos to deduce study/session")

    subject = get_unique(seqinfos, 'patient_id')
    locator = 'none_defined_yet'

    return {
        # TODO: request info on study from the JedCap
        'locator': locator,
        # Sessions to be deduced yet from the names etc TODO
        'session': 'not_working_yet',
        'subject': subject,
    }

def get_unique(seqinfos, attr):
    """Given a list of seqinfos, which must have come from a single study
    get specific attr, which must be unique across all of the entries

    If not -- fail!

    """
    values = set(getattr(si, attr) for si in seqinfos)
    assert (len(values) == 1)
    return values.pop()

def filter_dicom(dcmdata):
    """Return True if a DICOM dataset should be filtered out, else False"""
    comments = getattr(dcmdata, 'ImageComments', '')
    if len(comments):
        if 'reference volume' in comments.lower():
            print("Filter out image with comment '%s'" % comments)
            return True
    return False
    # Another format:return True if dcmdata.StudyInstanceUID in dicoms2skip else False


def filter_files(fn):
    """
    This is used by heudiconv to filter files based on the filename.
    The function returns a boolean for a given filename.
    """
    patterns_to_filter_out = ['README',
                              'requisition',
                              'realtime',
                              'edti',
                              'optional',
                              '3 plane loc',
                              'ASSET cal',
                              'clinical',
                              'plane_loc',
                             'nihpcasl']
#                               'rest_assetEPI']
    return all(pat not in fn for pat in patterns_to_filter_out)


def create_key(template, outtype=('nii.gz'), annotation_classes=None):
    if template is None or not template:
        raise ValueError('Template must be a valid format string')
    return template, outtype, annotation_classes


def infotodict(seqinfo, test_heuristics=False):
    ## sometimes seqinfo is a list of seqinfo objects, sometimes it is a seqinfo object
    if not hasattr(seqinfo,'keys'):
        seqinfo_dict = {'no_grouping' : seqinfo}
    else:
        seqinfo_dict = seqinfo
        
    mprage = create_key(
        'sub-{subject}/{session}/anat/sub-{subject}_{session}_acq-mprage_run-{item:03d}_T1w')
#     dti = create_key(
#         'sub-{subject}/{session}/dwi/sub-{subject}_{session}_run-{item:03d}_dwi')
    rest = create_key(
        'sub-{subject}/{session}/func/sub-{subject}_{session}_task-rest_run-{item:03d}_bold')
#     audition = create_key(
#         'sub-{subject}/{session}/func/sub-{subject}_{session}_task-audition_run-{item:03d}_bold')


    info = {mprage: [],
            rest: [],
#             audition: [],
           }
    heurs = {

        "('MPRAGE' in seq.series_description.upper().replace('_','').replace('-','').replace(' ',''))": "info[mprage].append([seq.series_id])",

        "('rest_assetEPI' in seq.series_description)": "info[rest].append([seq.series_id])",
#         "('Audition fmri' in seq.series_description)": "info[audition].append([seq.series_id])",

#         "('edti_cdiflist' in seq.series_description)": "info[dti].append([seq.series_id])",
    }
    
    if test_heuristics:
        for group_id, seqinfo in seqinfo_dict.items():
            for seq in seqinfo:
                for criterion, action in heurs.items():
                    eval(criterion)
                    eval(action)
                print("The defined heuristics evaluate")
                return None
    for group_id, seqinfo in seqinfo_dict.items():
        if len(seqinfo) > 30:
            print("There are a lot of entries provided here (%s)." 
                  " This heuristic file does not handle duplicate"
                  " series_id across the same accession_number."
                  " This can be avoided by passing subject/session"
                  " combinations individually to heudiconv"% len(seqinfo))
        for seq in seqinfo:
            for criterion, action in heurs.items():
                if eval(criterion):
                    eval(action)
                    break
        #

    return info


#### Confirm heuristic file fully evaluates

In [ ]:
heudiconv_helpers.helpers.dry_run_heurs(heuristics_script=heuristics_script,test_heuristics=True)
# ipdb.runcall(heudiconv_helpers.helpers.dry_run_heurs,heuristics_script=heuristics_script,seqinfo=seqinfo)

#### Validate the output of a heuristics file

In [ ]:
reload(heudiconv_helpers.helpers)
val = heudiconv_helpers.helpers.validate_heuristics_output(heuristics_script=heuristics_script,verbose = True)
print(val)

#### Examine the application of heuristics to the dataset

In [ ]:
## If using a specific heudiconv run use this:
dir_to_use = output_of_heudiconv_gen
# dir_to_use = Path('./bids_2017_07_14_generic/')

info_text_paths_gen = [x for x in dir_to_use.glob('**/info/*tsv')]
if len(info_text_paths_gen) == 0:
    info_text_paths_gen = [x for x in dir_to_use.glob('**/info/dicom*txt')]
if len(info_text_paths_gen) == 0:
    raise ValueError("Cannot find any info files")

In [ ]:
df_info_gen = pd.concat([pd.read_csv(p, sep = '\t').assign(file_path=p) for p in info_text_paths_gen]).reset_index(drop = True)

df_info_mapping = pd.concat(
    [
        heudiconv_helpers.helpers.dry_run_heurs(
            heuristics_script=heuristics_script,
            seqinfo=list(df.itertuples())) for x,df in df_info_gen.groupby('file_path') ],
axis = 0)
df_info_mapping = df_info_mapping.assign(file_path = lambda df: df.file_path.astype(str))
df_info_mapping = df_info_mapping.assign(participant_id = lambda df: 'sub-' + df.file_path.str.extract('.*.heudiconv/(?P<subj>\d{4})/.*', expand = True))
df_info_mapping.to_csv(info_mapping_csv,index=False,sep=',')
df_info_mapping.head()

In [ ]:
# Some code for debugging
# reload(heudiconv_helpers.helpers)
# for x, df in df_info_gen.groupby('file_path'):
#     seqinfo = list(df.itertuples())
#     seqinfo_dict = {x : seqinfo}
# break
# heudiconv_helpers.helpers.dry_run_heurs(heuristics_script=heuristics_script,seqinfo=seqinfo_dict)
# ipdb.runcall(heudiconv_helpers.helpers.dry_run_heurs,
#              heuristics_script=heuristics_script,
#              seqinfo=seqinfo)

###  Generate heudiconv swarm commands

In [ ]:
reload(heudiconv_helpers.helpers)
df_sing = df_bids
df_sing['cmd'] = df_bids.apply(
    lambda row:
    heudiconv_helpers.helpers.make_heud_call(row = row,
                           project_dir = project_dir_absolute,
                           output_dir=outdir,
                           conversion = True,
                           container_image = sing_image,
                           use_scratch = True,
                            grouping = 'studyUID',
                           heuristics_script = heuristics_script,
                          dev_dir = heudiconv_source_code,
                          dev = True
                      
                            ),
        axis = 1)

run_type = 'conversion'
conversion_dict[run_type] = Path(scripts_dir).joinpath(f'heudiconv_{run_type}_{analysis_version}.cmd')

# not all commands resolve to a single dicom so getting unique ones before writing swarm
conversion_dict[run_type].write_text('\n'.join(df_sing.cmd.drop_duplicates())) 
print([x for x in conversion_dict[run_type].read_text().splitlines()][0])

In [ ]:
(len(df_sing.cmd),len(df_sing.cmd.drop_duplicates()))

### Run heudiconv conversion

Example of running the commands on a server:

In [ ]:
Example of running the commands on nih cluster:

# Other ways of processing the data when uploading to public repositories.

## Fixing participants tsv

In [ ]:
import bids
layout = bids.BIDSLayout(output_of_heudiconv.as_posix())
df_pybids = layout.as_data_frame().query('path.str.contains("nii.gz")')

In [ ]:
df_pybids['json_path'] = (
    df_pybids.path.apply(
        lambda x: Path(''.join([*x.split('.')[:-2], '.json']))))

df_pybids.head()

## Remove ttl files

In [ ]:
prov_dir = Path('conversion_provenance')
if not prov_dir.exists(): prov_dir.mkdir()
!find bids_2018_08 -name '*ttl' -exec mv {} conversion_provenance \;

## Write text files

In [ ]:
description_file = output_of_heudiconv.joinpath('dataset_description.json')
description_file.touch()
description_file.write_text("""\
{
    "Name": "NNDSP Data",
    "BIDSVersion": "1.0.1",
    "Authors": [
        "John A. Lee",
        "Dylan Nielson",
        "Adam Thomas"
    ],
    "Funding": "NIMH Intramural Research Program"
}
""")

In [ ]:
task_dict = {'rest' : 'Resting State'}
for k,v in task_dict.items():

    json_path = list(output_of_heudiconv.glob('task*' + k + '*.json'))
    assert len(json_path) == 1
    json_path = json_path[0]
    row = pd.Series({'json_path' : json_path})
    heudiconv_helpers.helpers.json_action(row,
                                          fieldnames = ['TaskName'],
                                          action = 'set',
                                          values_to_set = [v])

In [ ]:
%%writefile path_to_bids_directory/README
# Overview
This dataset is released as part of the NIMH/NHGRI Data-Sharing Project (NNDSP). 

# Summary
The dataset contains 441 subjects (188 females) with an age range of 5-77 years old (median = 20 years). At least one anatomical MPRAGE T1w scan was collected for each subject with a total of 490 scans. Each anatomical scan was rated from 1 to 4 with a score of 3 or more being considered unusable as described by Blumenthal et al. (2002). Using this threshold, 32  of the 487 scans were rated as bad by the human raters. Resting state fMRI scans were collected for over 80% of the subjects and task-based fMRI scans (an audition task) were collected in over 30% of the subjects.

## Data Acquisition
Data was acquired on GE's 3T, Signa_HDxt and Signa_HDx scanners



In [ ]:
output_of_heudiconv.joinpath('dataset_description.json').read_text()

If sharing the dataset broadly this is a good time to deface the relevant scans...

## Remove scans with bad metadata

## Jitter scan date

In [ ]:
def get_scans_tsv(series,path_col):
    scans_tsv = Path(series[path_col]).parent.parent.joinpath('sub-' + series.subject + '_' + 'ses-' + series.session + '_scans.tsv')
    
    return scans_tsv

df_pybids['scans_tsv_path'] = (
    df_pybids.
                apply(lambda df:
                      get_scans_tsv(df,'path'),
                      axis = 1)
)

In [ ]:
df_offset = (heudiconv_helpers.helpers.gen_subj_time_jitter(
    df_pybids['subject'].
    dropna().
    drop_duplicates().
    values,
    acq_date_time_offset)
            )

In [ ]:
df_tsvs =  df_pybids.drop_duplicates('scans_tsv_path',keep='first')
df_tsvs.apply(lambda row: 
              heudiconv_helpers.helpers.rewrite_tsv(
                  row['scans_tsv_path'],
                  df_offset,row['subject'],
                            dry_run=False),
              axis = 1)



## Remove events files for rest

In [ ]:
find_arg = '{}'
!find {output_of_heudiconv} -name '*rest*events*' -exec rm {find_arg} \;

## Remove some json fields including ones containing PII

In [ ]:
df_pybids = heudiconv_helpers.helpers.get_bids_df(output_of_heudiconv,scans_only=True)
df_pybids['participant_id'] = 'sub-' + df_pybids.subject

In [ ]:
# !chmod -R 770 {output_of_heudiconv}
fieldname_tuples = [
    ('global','const','AccessionNumber'),
    ('global','const','RequestAttributesSequence'),
    ('global','const','SeriesTime'),
    ('global','const','StudyID'),
    ('global','const','StudyTime'),
    ('global','const','ContentTime'),
    'AcquisitionDateTime',
    'InstitutionAddress']

df_pybids.apply(
    lambda row: 
    heudiconv_helpers.helpers.json_action(
        row,fieldnames = fieldname_tuples,action='delete'),
    axis = 1);

In [ ]:
df_all_files = heudiconv_helpers.helpers.get_bids_df(output_of_heudiconv)
for x,df in df_all_files.query(f'path.str.contains("{output_of_heudiconv}/task-")').iterrows():
    (heudiconv_helpers.helpers.
     json_action(
         df,action='delete',json_col='path', fieldnames= ["CogAtlasID"])
    )

## Bids validation

In [ ]:
reload(heudiconv_helpers.helpers)
print(heudiconv_helpers.helpers.validate_bids_dir(output_of_heudiconv,verbose=True))


## Final tidy up

In [ ]:
hidden = output_of_heudiconv / '.heudiconv'
if hidden.exists():
    target = project_dir_absolute.joinpath(f'.{analysis_version}_conversion')
    if not target.exists():
        target.mkdir()
    !mv {hidden} {target}



In [ ]:
todos = !find {output_of_heudiconv} -name '*.json' |xargs grep TODO
if todos:
    print(todos)
    print("Some of the files contain TODOs")
    
if any(
    [Path(output_of_heudiconv).joinpath('.heudiconv').exists(),
    Path('bids/sourcedata').exists()]):
    print('move files')

